In [1]:
# RNN(Long Short-Term Memory, LSTM)

In [13]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [14]:
def mergeData():
    SaveFile_Name = 'data.csv'
    file_list = os.listdir('data')
    df = pd.read_csv('data'+'\\'+file_list[0])
    df.to_csv(SaveFile_Name,encoding="utf_8_sig",index=False)
    for i in range(1,len(file_list)):
        df = pd.read_csv('data'+'\\'+file_list[i])
        df.to_csv(SaveFile_Name,encoding="utf_8_sig",index=False, header=False, mode='a+')

In [15]:
def readData():
    train = pd.read_csv("data.csv")
    return train

In [33]:
def changeYear(data):
    for i in range(0,data.shape[0]):
        Date=data["日期"][i].split('/')
        year,month,date=Date[0],Date[1],Date[2]
        year=str(int(year)+1911)
        data.loc[i,"日期"]=year+'/'+month+'/'+date
    return data

In [34]:
# Augment Features
def augFeatures(data):
  data["日期"] = pd.to_datetime(data["日期"])
  data["年"] = data["日期"].dt.year
  data["月"] = data["日期"].dt.month
  data["日"] = data["日期"].dt.day
  data["第幾日"] = data["日期"].dt.dayofweek
  return data

In [35]:
def normalize(data):
    for i in range(0,data.shape[0]):
        if data["漲跌價差"][i]=='X0.00':
            data.loc[i,"漲跌價差"]=str(int(data["收盤價"][i])-int(data["收盤價"][i-1]))
    data=data.drop(["日期"], axis=1)
    data=data.drop(["成交股數"], axis=1)
    data=data.drop(["成交金額"], axis=1)
    data=data.drop(["漲跌價差"], axis=1)
    data=data.drop(["成交筆數"], axis=1)
    data=data.convert_objects(convert_numeric=True)
    datanormalize=data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return datanormalize

In [42]:
def buildTrain(train, pastDay=30, futureDay=5):
  X_train, Y_train = [], []
  for i in range(train.shape[0]-futureDay-pastDay):
    X_train.append(np.array(train.iloc[i:i+pastDay]))
    Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["開盤價"]))
  return np.array(X_train), np.array(Y_train)

In [43]:
def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

In [44]:
# 將Training Data取一部份當作Validation Data
def splitData(X,Y,rate):
  X_train = X[int(X.shape[0]*rate):]
  Y_train = Y[int(Y.shape[0]*rate):]
  X_val = X[:int(X.shape[0]*rate)]
  Y_val = Y[:int(Y.shape[0]*rate)]
  return X_train, Y_train, X_val, Y_val

In [45]:
def buildOneToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
  # output shape: (1, 1)
  model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [48]:

mergeData()
train=readData()
train=changeYear(train)
train=augFeatures(train)
train=normalize(train)
train_x, train_y = buildTrain(train, 1, 1)
train_x, train_y = shuffle(train_x, train_y )

# split training data and validation data
train_x, train_y , val_x, val_y = splitData(train_x, train_y , 0.1)
train_y = train_y[:,np.newaxis]
val_y = val_y[:,np.newaxis]

model = buildOneToOneModel(train_x.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")

model.fit(train_x, train_y, epochs=1000, batch_size=128, validation_data=(val_x, val_y), callbacks=[callback])


C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(1, 8), return_sequences=True)`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 1, 10)             760       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 1, 1)              11        
Total params: 771
Trainable params: 771
Non-trainable params: 0
_________________________________________________________________
Train on 5571 samples, validate on 618 samples
Epoch 1/1000
5571/5571 [==============================] - 1s 132us/step - loss: 0.0331 - val_loss: 0.0224
Epoch 2/1000
5571/5571 [==============================] - 0s 11us/step - loss: 0.0135 - val_loss: 0.0066
Epoch 3/1000
5571/5571 [==============================] - 0s 11us/step - loss: 0.0030 - val_loss: 9.6957e-04
Epoch 4/1000
5571/5571 [==============================] - 0s 9us/step - loss: 5.3603e-04 - val_loss: 3.1170e-04
Epoch 5/1000
5571/5571 [==============================] - 0s 

Epoch 68/1000
5571/5571 [==============================] - 0s 9us/step - loss: 1.1438e-04 - val_loss: 6.5389e-05
Epoch 69/1000
5571/5571 [==============================] - 0s 9us/step - loss: 1.1385e-04 - val_loss: 6.3950e-05
Epoch 70/1000
5571/5571 [==============================] - 0s 9us/step - loss: 1.1344e-04 - val_loss: 6.4038e-05
Epoch 71/1000
5571/5571 [==============================] - 0s 10us/step - loss: 1.1408e-04 - val_loss: 6.4776e-05
Epoch 72/1000
5571/5571 [==============================] - 0s 9us/step - loss: 1.1318e-04 - val_loss: 6.3693e-05
Epoch 73/1000
5571/5571 [==============================] - 0s 9us/step - loss: 1.1275e-04 - val_loss: 6.4754e-05
Epoch 74/1000
5571/5571 [==============================] - 0s 9us/step - loss: 1.1343e-04 - val_loss: 6.4569e-05
Epoch 75/1000
5571/5571 [==============================] - 0s 10us/step - loss: 1.1352e-04 - val_loss: 6.5678e-05
Epoch 76/1000
5571/5571 [==============================] - 0s 10us/step - loss: 1.1253e-04 - v

5571/5571 [==============================] - 0s 9us/step - loss: 1.0480e-04 - val_loss: 5.8209e-05
Epoch 140/1000
5571/5571 [==============================] - 0s 9us/step - loss: 1.0273e-04 - val_loss: 5.5314e-05
Epoch 141/1000
5571/5571 [==============================] - 0s 9us/step - loss: 1.0250e-04 - val_loss: 5.6000e-05
Epoch 142/1000
5571/5571 [==============================] - 0s 9us/step - loss: 1.0299e-04 - val_loss: 5.6068e-05
Epoch 143/1000
5571/5571 [==============================] - 0s 10us/step - loss: 1.0181e-04 - val_loss: 5.8658e-05
Epoch 144/1000
5571/5571 [==============================] - 0s 10us/step - loss: 1.0311e-04 - val_loss: 5.7503e-05
Epoch 145/1000
5571/5571 [==============================] - 0s 9us/step - loss: 1.0303e-04 - val_loss: 5.6124e-05
Epoch 146/1000
5571/5571 [==============================] - 0s 9us/step - loss: 1.0207e-04 - val_loss: 5.8126e-05
Epoch 147/1000
5571/5571 [==============================] - 0s 10us/step - loss: 1.0261e-04 - val_los